In [32]:
import torch_geometric
import torch
import networkx as nx
from typing import Optional, List, Any
from torch_geometric.utils.convert import from_networkx
from heapq import heapify
import random

In [13]:
def get_parent(i: int) -> Optional[int]:
    if i <= 0:
        return None
    return (i - 1) // 2

def is_heap(l) -> bool:
    for (i, e) in enumerate(l):
        parent = get_parent(i)
        if parent is not None and l[parent] > e:
            return False
    return True

In [45]:
def generate_point(token_dim: int = 10, min_len: int = 1, max_len: int = 32, p_heapify: float = 0.4) -> List[int]:
    result = []
    for i in range(0, random.randint(min_len, max_len)):
        result.append(list(torch.randn(token_dim)))
    if random.random() < p_heapify: 
        heapify(result)
    return torch.tensor(result)

In [46]:
generate_point()

tensor([[-0.7159, -0.9623,  1.1429,  0.8942,  0.8918, -0.4359, -1.6203,  0.2658,
         -1.5762, -0.8939],
        [-0.3626, -1.1565,  0.2292,  0.6901, -0.5968, -1.4608,  1.1956, -0.7660,
          0.8889,  0.3868],
        [ 0.3187,  0.1426,  0.2522,  0.6110, -0.0065,  0.2905, -0.1317,  0.2203,
          0.4433, -1.0786],
        [ 0.1506, -1.1281, -1.8438, -0.0350, -0.4874, -0.0377, -0.1447,  1.8745,
         -0.6931, -1.9233],
        [-0.3580,  0.6143, -0.1700, -1.0301,  0.3262, -0.2034, -0.7491,  0.3223,
          0.6436,  0.5754],
        [ 1.0705,  1.0931,  0.2554,  2.7021, -1.3401, -0.5866, -0.4582, -0.9610,
          1.4302,  1.4942],
        [ 0.8337,  0.3894, -1.2481,  0.3478,  0.0849,  0.2671,  0.9656,  0.8241,
         -1.5052, -0.3029],
        [ 1.0952,  0.0428,  0.9180,  0.5080, -0.0261,  0.5001,  1.5315, -0.7833,
         -0.6327, -0.4572]])

In [49]:
def generate_data(n: int, token_dim: int = 10, min_len: int = 1, max_len: int = 32, p_heapify: float = 0.4):
    for _ in range(0, n):
        yield generate_point(token_dim, min_len, max_len, p_heapify)